# AFML Part 1 - Team 44_XLR8 (BEST VERSION)
## Optimized for NMSE < 0.1

**Strategy**: Multiple improvements for best performance

In [ ]:
# Mount Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir('/content/drive/MyDrive/AFML_KAAGLE')
    print("✓ Colab")
except:
    print("✓ Local")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## Load Data

In [ ]:
print("Loading data...")
train_clean = pd.read_csv('train-part1-clean.csv').values.astype(np.float32)
train_noise = pd.read_csv('train-part1-noise.csv').values.astype(np.float32)
test_data = pd.read_csv('test-part1.csv').values.astype(np.float32)

print(f"Clean: {train_clean.shape}")
print(f"Noisy: {train_noise.shape}")
print(f"Test: {test_data.shape}")

## Preprocessing with RobustScaler (Better for Outliers)

In [ ]:
# Split with more validation data for better NMSE estimate
X_train, X_val, y_train, y_val = train_test_split(
    train_noise, train_clean, test_size=0.15, random_state=42
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# Use RobustScaler (better for noisy data)
scaler_X = RobustScaler()
scaler_y = RobustScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
test_scaled = scaler_X.transform(test_data)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)

# Convert to tensors
X_train_t = torch.FloatTensor(X_train_scaled).to(device)
y_train_t = torch.FloatTensor(y_train_scaled).to(device)
X_val_t = torch.FloatTensor(X_val_scaled).to(device)
y_val_t = torch.FloatTensor(y_val_scaled).to(device)
test_t = torch.FloatTensor(test_scaled).to(device)

print("✓ Preprocessing done")

## Improved Model with Skip Connections

In [ ]:
class DenoisingNet(nn.Module):
    def __init__(self, input_dim=20):
        super().__init__()
        
        # Encoder
        self.enc1 = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )
        
        self.enc2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )
        
        self.enc3 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )
        
        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1)
        )
        
        # Decoder with skip connections
        self.dec1 = nn.Sequential(
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )
        
        self.dec2 = nn.Sequential(
            nn.Linear(256 + 256, 512),  # Skip from enc3
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )
        
        self.dec3 = nn.Sequential(
            nn.Linear(512 + 256, 256),  # Skip from enc1
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )
        
        self.output = nn.Linear(256, input_dim)
    
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        
        # Bottleneck
        b = self.bottleneck(e3)
        
        # Decoder with skip connections
        d1 = self.dec1(b)
        d2 = self.dec2(torch.cat([d1, e3], dim=1))  # Skip
        d3 = self.dec3(torch.cat([d2, e1], dim=1))  # Skip
        
        out = self.output(d3)
        return out

model = DenoisingNet().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## Training Setup - Optimized

In [ ]:
BATCH_SIZE = 1024  # Larger for stability
NUM_EPOCHS = 150   # More epochs
LR = 0.002         # Higher initial LR

train_loader = DataLoader(
    TensorDataset(X_train_t, y_train_t),
    batch_size=BATCH_SIZE, shuffle=True, drop_last=True
)

val_loader = DataLoader(
    TensorDataset(X_val_t, y_val_t),
    batch_size=BATCH_SIZE, shuffle=False
)

# Huber loss (more robust to outliers)
criterion = nn.SmoothL1Loss()

# AdamW optimizer
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

# Cosine annealing with warm restarts
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=20, T_mult=2, eta_min=1e-6
)

print(f"Batches per epoch: {len(train_loader)}")

## Training Loop

In [ ]:
best_val_loss = float('inf')
train_losses, val_losses = [], []
patience_counter = 0
PATIENCE = 30

for epoch in range(NUM_EPOCHS):
    # Train
    model.train()
    train_loss = 0
    
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False):
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validate
    model.eval()
    val_loss = 0
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            pred = model(X_batch)
            loss = criterion(pred, y_batch)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    scheduler.step()
    
    # Save best
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
        print(f"✓ Epoch {epoch+1}/{NUM_EPOCHS} - Train: {train_loss:.6f}, Val: {val_loss:.6f} [SAVED]")
    else:
        patience_counter += 1
        if (epoch+1) % 10 == 0:
            print(f"  Epoch {epoch+1}/{NUM_EPOCHS} - Train: {train_loss:.6f}, Val: {val_loss:.6f}")
    
    # Early stopping
    if patience_counter >= PATIENCE:
        print(f"\nEarly stopping at epoch {epoch+1}")
        break

print(f"\nBest val loss: {best_val_loss:.6f}")

## Ensemble Prediction (Average 3 Models)

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Predict with test-time augmentation (TTA)
def predict_with_tta(model, data_tensor, n_tta=5):
    predictions = []
    
    with torch.no_grad():
        # Original prediction
        pred = model(data_tensor).cpu().numpy()
        predictions.append(pred)
        
        # TTA with small noise
        for _ in range(n_tta - 1):
            noise = torch.randn_like(data_tensor) * 0.01
            pred = model(data_tensor + noise).cpu().numpy()
            predictions.append(pred)
    
    return np.mean(predictions, axis=0)

print("Predicting with TTA...")
test_pred_scaled = predict_with_tta(model, test_t, n_tta=5)
val_pred_scaled = predict_with_tta(model, X_val_t, n_tta=5)

# Inverse transform
test_pred = scaler_y.inverse_transform(test_pred_scaled)
val_pred = scaler_y.inverse_transform(val_pred_scaled)

## Calculate NMSE

In [ ]:
mse = np.mean((y_val - val_pred) ** 2)
variance = np.var(y_val)
nmse = mse / variance

print(f"\n{'='*70}")
print(f"VALIDATION NMSE: {nmse:.6f}")
print(f"Target: < 0.3")
print(f"{'='*70}")

if nmse < 0.1:
    print(f"🎉 EXCELLENT! NMSE < 0.1!")
elif nmse < 0.3:
    print(f"✅ SUCCESS! NMSE < 0.3!")
else:
    print(f"⚠️  NMSE > 0.3 (but should still be good)")

print(f"\nMSE: {mse:.6f}")
print(f"Variance: {variance:.6f}")

## Save Submission

In [ ]:
submission = pd.DataFrame(test_pred)
submission.to_csv('submission.csv', index=False)

print("\n✓ Saved: submission.csv")
print(f"Shape: {submission.shape}")
print(f"\nFirst few rows:")
submission.head()

## Plot Results

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train', alpha=0.7)
plt.plot(val_losses, label='Val', alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Plot actual vs predicted for validation
plt.scatter(y_val.flatten()[:1000], val_pred.flatten()[:1000], alpha=0.3, s=1)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title(f'Predictions (NMSE={nmse:.4f})')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Summary:")
print(f"Best Val Loss: {best_val_loss:.6f}")
print(f"NMSE: {nmse:.6f}")
print(f"Target: < 0.3")

## Next Steps

1. ✅ Upload `submission.csv` to Kaggle
2. ✅ Share this notebook with all 6 TAs
3. ✅ Use `submission.csv` in Part 2